In [ ]:
import numpy as np
from numpy import sqrt, exp
import time
import matplotlib.pyplot as plt
import numba


#########################################################################################
#    Input parameters for the asymmetric potential
#########################################################################################

lamda = 10**(-17)
beta = 10**(-14)


################################################################################################################
#     dN stepsize of e-fold time variable to solve stochastic differential equations
#     phi_in = initial value of inflaton field to generate desired number of e-folds for inflation to take place
#################################################################################################################


dN = 0.001

@numba.njit
def V(x):
    return (lamda/24)*x**4+(beta/6)*x**3

@numba.njit
def dV(x):
    return (lamda/6)*x**3+(beta/2)*x**2

@numba.njit
def ddV(x):
    return (lamda/2)*x**2+(beta)*x

#########################################################################################################
#     This code solve the background evolution of stochastic differential equations without noise terms.
#########################################################################################################

@numba.njit
def back_evolve(phi_in, efolds):
    N_test = efolds
    n = int(N_test/dN)
    phi = np.zeros(n); phi[0] = phi_in
    phi_M = np.zeros(n); phi_M[0] = -dV(phi_in)/V(phi_in)

    for i in range(n-1):
        K1 = dN*phi_M[i]
        L1 = dN*( -3*phi_M[i] + 0.5*phi_M[i]**3 - (3 - 0.5*phi_M[i]**2)*dV(phi[i])/V(phi[i]) )
        K2 = dN*(phi_M[i] + L1)
        L2 = dN*( -3*(phi_M[i] + L1) + 0.5*(phi_M[i] + L1)**3 - (3 - 0.5*(phi_M[i] + L1)**2)*dV(phi[i] + K1)/V(phi[i] + K1) )

        phi[i+1] = phi[i] + 0.5*(K1 + K2)
        phi_M[i+1] = phi_M[i] + 0.5*(L1 + L2)

    return phi, phi_M

################################################################################################
#     In this code simulations will be performed for approximated slow-roll noise amplitudes.
################################################################################################

@numba.njit(parallel=True)
def field_correlations(I, phi_, dphi_, efolds_):
    N_end = efolds_
    N = np.linspace(0, N_end, int(N_end/dN))
    phi_bg = phi_; dphi_bg = dphi_ 

    eps1 = 0.5*dphi_bg**2
    H = sqrt(V(phi_bg)/(3 - eps1))
    delphi2 = np.zeros(len(N))

##################################################################################################################
#      RK-2 method will be employed to solve the stochastic differential equations
##################################################################################################################

    
    for i in numba.prange(I):

        F = np.random.randn(len(N))/sqrt(dN)
        S = np.random.choice(np.array([-1, 1]), len(N))

       

        

        phi_cg = np.zeros(len(N))
        dphi_cg = np.zeros(len(N))
        phi_cg[0] = phi_bg[0]
        dphi_cg[0] = dphi_bg[0]


        for j in range(len(N)-1):

            k1 = dN*dphi_cg[j] + (dN*F[j] - S[j]*sqrt(dN))*H[j]/(2*np.pi)
            l1 = dN*( -3*dphi_cg[j] + 0.5*dphi_cg[j]**3 - (3 - 0.5*dphi_cg[j]**2)*dV(phi_cg[j])/V(phi_cg[j]) )
            k2 = dN*(dphi_cg[j] + l1) + (dN*F[j] + S[j]*sqrt(dN))*H[j+1]/(2*np.pi)
            l2 = dN*( -3*(dphi_cg[j] + l1) + 0.5*(dphi_cg[j] + l1)**3 - (3 - 0.5*(dphi_cg[j] + l1)**2) \
                     *dV(phi_cg[j] + k1)/V(phi_cg[j] + k1) )

            phi_cg[j+1] = phi_cg[j] + 0.5*(k1 + k2)
            dphi_cg[j+1] = dphi_cg[j] + 0.5*(l1 + l2)

        delphi = phi_cg - phi_bg
        delphi2 += delphi**2

    delphi2 = delphi2/I

    return delphi2

##################################################################################################################

    ti = time.time()

##################################################################################################################

    N_end = 70
    N = np.linspace(0, N_end, int(N_end/dN))
    phi_bg, dphi_bg = back_evolve(23.19, N_end)
    nsim = 10**5
    delphi2 = field_correlations(nsim, phi_bg, dphi_bg, N_end)

    eps1 = 0.5*dphi_bg**2
    eps2 = np.gradient(eps1, dN)/eps1
    H = sqrt(V(phi_bg)/(3-eps1))
    d_delphi2 = np.gradient(delphi2, dN)
    P_zeta_stochastic = ( 0.5/(eps1*(1-eps1)) )*(d_delphi2 - eps2*delphi2) 
    P_zeta = H**2 /(8*np.pi**2 *eps1) 
#################################################################################################################
#      P_zeta_stochastic - Stochastic power spectrum
#      P_zeta            - Slow roll power spectrum
#      nsim              - number of simulations, we will perform 10^5 simulations for our inflaton potential
##################################################################################################################    

    tf = time.time()
    print("Code execution time for " + str(nsim) + "simulations: " + str(tf-ti) + " seconds.")
##################################################################################################################

    plt.scatter(N, P_zeta_stochastic, s=2)
    plt.plot(N, P_zeta, c='r')
    plt.yscale('log')
    plt.show()

In [ ]:
##################################################################################################
#    To find the power spectrum and spectral index at pivot scale
##################################################################################################

P_zeta_stochastic[10000],(1+(1/(1-eps1[:60000]))*((np.gradient(P_zeta_stochastic,10)))/(P_zeta_stochastic))[10000]